<a href="https://colab.research.google.com/github/anablima/MBA_DataEngineer/blob/main/Bootcamp_Data_Engineering_Exercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Temos duas bases de dados pessoais (fictícios)


*   cadastro.csv (dados de cadastro)
*   dados_medicos.csv (registros médicos)

Temos que unificar as duas bases e criar uma nova base agrupada

Código localizado em https://github.com/renzoziegler/data_engineering_bootcamp


Exercício 1:


*   Importar o Pandas
*   Instalar e importar a biblioteca fuzzywuzzy (será importante mais pra frente)
*   Ler os arquivos CSV fornecidos
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/cadastro.csv
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/dados_medicos.csv
*   Realizar uma análise exploratória dos dados



In [1]:
# Instalando e importando libs necessárias
!pip install fuzzywuzzy
import pandas as pd
import fuzzywuzzy

In [ ]:
# Importando dados de cadastro
cad = pd.read_csv("https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/cadastro.csv", sep=",")
data_med = pd.read_csv("https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/dados_medicos.csv", sep=",")

Para análise exploratória, vamos entender as bases de dados: seu tamanho, quais dados estão disponíveis, qual o formato deles, se estão completos ou não.....

In [5]:
# Exibindo primeiras linhas do dataframe
cad.head()

,nome,data_nasc,cpf,mae,celular,email,telefone_fixo
0,Cauã Isaac Caldeira,02-03-1945,35083528118,Vitória Kamilly Sueli,989048340,cauaisaaccaldeira__cauaisaaccaldeira@hardquali...,28317237
1,Lúcia Carolina Luzia Almada,11-11-1946,6480417194,Stefany Teresinha,997275887,luciacarolinaluziaalmada_@santarte.com,27446850
2,Vicente Marcos Edson Castro,15-01-1959,72192158432,Cecília Laura Larissa,994560743,vicentemarcosedsoncastro-74@lavorosjc.com.br,28581372
3,Yago Pedro Henrique da Cruz,24-10-1945,68328310562,Cristiane Pietra,982340551,yagopedrohenriquedacruz_@elevaconsultancy.com.br,25189906
4,Valentina Stefany Alana Cardoso,23-03-1953,76298423761,Francisca Adriana,999976430,valentinastefanyalanacardoso..valentinastefany...,26584091


In [8]:
# Obtendo informações sobre o DataFrame
cad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nome           393 non-null    object
 1   data_nasc      393 non-null    object
 2   cpf            393 non-null    int64 
 3   mae            393 non-null    object
 4   celular        393 non-null    int64 
 5   email          393 non-null    object
 6   telefone_fixo  393 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 21.6+ KB


In [9]:
# Resumo das colunas numéricas
cad.describe()

,cpf,celular,telefone_fixo
count,3.930000e+02,3.930000e+02,3.930000e+02
mean,5.023708e+10,1.074284e+09,4.058497e+07
std,2.744231e+10,9.593948e+08,9.629149e+07
min,2.004566e+08,9.811014e+08,2.501665e+07
25%,2.922306e+10,9.856004e+08,2.739774e+07
50%,5.149845e+10,9.894505e+08,2.984399e+07
75%,7.353482e+10,9.950997e+08,3.732729e+07
max,9.957899e+10,1.199897e+10,1.135495e+09


In [11]:
# Verificando se há valores nulos
cad.isnull().sum()

,0
nome,0
data_nasc,0
cpf,0
mae,0
celular,0
email,0
telefone_fixo,0


In [6]:
# Exibindo primeiras linhas do dataframe
data_med.head()

,nome,data_nasc,cpf,tipo_sanguineo,peso,altura
0,Alice Liz Adriana Duarte,22/10/1975,58548099313,O-,89,"1,57"
1,Kaue César Lima,01/01/1900,31515413217,AB-,62,"1,99"
2,Otávio Felipe Henry Almeida,10/01/1951,58271711156,A-,85,"1,66"
3,Rafaela Francisca Mendes,07/11/1974,31087464641,O+,46,"1,74"
4,Bryan Joaquim Roberto Rosa,07/09/1957,61606247174,AB-,66,"1,67"


In [12]:
# Obtendo informações sobre o DataFrame
data_med.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   nome            368 non-null    object
 1   data_nasc       368 non-null    object
 2   cpf             368 non-null    int64 
 3   tipo_sanguineo  368 non-null    object
 4   peso            368 non-null    int64 
 5   altura          368 non-null    object
dtypes: int64(2), object(4)
memory usage: 17.4+ KB


In [15]:
# Resumo das colunas numéricas
data_med.describe()

,cpf,peso
count,3.680000e+02,368.000000
mean,4.060608e+10,73.105978
std,3.105120e+10,17.110227
min,0.000000e+00,13.000000
25%,7.943746e+09,59.000000
50%,4.084276e+10,71.500000
75%,6.525570e+10,85.000000
max,9.957899e+10,110.000000


In [16]:
# Verificando a quantidade de valores únicos em cada coluna
data_med.nunique()

,0
nome,368
data_nasc,297
cpf,302
tipo_sanguineo,8
peso,67
altura,53


In [17]:
# Verificando se há valores nulos
data_med.isnull().sum()

,0
nome,0
data_nasc,0
cpf,0
tipo_sanguineo,0
peso,0
altura,0


Exercício 2: Pré-processamento
*   Quais tarefas vocês listam como necessárias?
 *  Padronização de campos
 *  Substituir valores
 *  Retirar caracteres especiais
*   Listar e executar!

In [ ]:
#Seu código aqui

Exercício 3: Indexação
*    Listar possíveis blocking Keys
*    Definir uma a ser utilizada
*    Criar os blocos
*    Listar os possíveis matches em cada bloco

In [ ]:
#Seu código aqui

Exercício 4: Comparação par a par
*    Desenvolver um algoritmo de comparação de cada match potencial do exercício anterior
*    Definir uma nota de similaridade para cada comparação
 *   Para cada parâmetro

In [ ]:
#Seu código aqui

Exercício 5: Classificação
*    Classificar cada par, baseado num threshold
*    Definir um peso para cada parâmetro

In [ ]:
#Seu código aqui

Exercício 6: Avaliação
*     Buscar uma amostra das classificações (de 10 a 20 registros)
*     Classificar cada uma em TP, TN, FP, FN
*     Calcular acurácia, precisão e recall

In [ ]:
#Seu código aqui